In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun  8 13:38:27 2018

@author: ALSSHXU
"""#
# -*- coding: utf-8 -*-
"""
Created on Fri Jun  8 10:51:24 2018

@author: ALSSHXU
"""

# -*- coding: utf-8 -*-
"""
Created on Wed Jun  6 14:54:14 2018

@author: ALSSHXU
"""

# -*- coding: utf-8 -*-
"""
Created on Mon May  7 21:25:40 2018

@author: UX390-GS048T
"""
import pickle
import numpy as np
import matplotlib.pyplot as plt 
from skimage.color import rgb2gray
import os
import cv2

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

#from PIL import Image

#def rgb2gray(rgb):
#    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def Main():
    
    train_data_sizes = []
    train_data_coords = []
    train_data_features = []
    train_data_labels = []
    
    try:
        with open('F:/Udacity/Project 2/traffic-signs-data/train.p','rb') as snapFile:
        #with open('C:/Users/alsshxu/Desktop/2018 HX/UdaCity/Project 2/traffic-signs-data/train.p','rb') as snapFile:
            savedSnaps = pickle.load(snapFile)
    except IOError as e:
        return False
    #print(savedSnaps)
    train_data_sizes=savedSnaps['sizes']
    train_data_coords = savedSnaps['coords']
    train_data_features = savedSnaps['features']
    train_data_labels = savedSnaps['labels']
    train_images = rgb2gray(train_data_features)
    
    #print (train_data_sizes)
    #print (train_data_coords[0][0])
    #print (train_data_features)
    #print (train_data_sizes[0][0])

    for i in range(0,34799):
        train_data_coords[i][0]=int(train_data_coords[i][0] * 32 / train_data_sizes[i][0])
        train_data_coords[i][1]=int(train_data_coords[i][1] * 32 / train_data_sizes[i][1])
        train_data_coords[i][2]=int(train_data_coords[i][2] * 32 / train_data_sizes[i][0])+1
        train_data_coords[i][3]=int(train_data_coords[i][3] * 32 / train_data_sizes[i][1])+1
        crop_img = train_images[i][train_data_coords[i][1]:train_data_coords[i][3], train_data_coords[i][0]:train_data_coords[i][2]]
        train_images[i] = cv2.resize(crop_img,(32,32))
    name_values = np.genfromtxt('signnames.csv', skip_header=1, dtype=[('myint','i8'), ('mysring','S55')], delimiter=',')
    train_data_sizes = []
    train_data_coords = []
    """
    print(train_data_sizes)
    a1,b1 = np.shape(train_data_sizes)
    print (a1)
    print (b1)
    
    a2,b2 = np.shape(train_data_coords)
    print (a2)
    print (b2)

    a3,b3,c3,d3 = np.shape(train_data_features)
    print (a3)
    print (b3)
    print (c3)
    print (d3)

    a4 = np.shape(train_data_labels)
    print (a4)
    """
    
    plt.hist(train_data_labels, 42)
    plt.show()
    
    # Import the `pyplot` module of `matplotlib`
    # Determine the (random) indexes of the train_images that you want to see 
    traffic_signs = [1, 200, 3000, 6000]
    
    # Fill out the subplots with the random train_images that you defined 
    for i in range(len(traffic_signs)):
        plt.subplot(1, 4, i+1)
        plt.axis('off')
        plt.imshow(train_images[traffic_signs[i]], cmap='gray')
        plt.subplots_adjust(wspace=0.5)
        plt.show()
        print("shape: {0}, min: {1}, max: {2}".format(train_images[traffic_signs[i]].shape, 
                                                  train_images[traffic_signs[i]].min(), 
                                                  train_images[traffic_signs[i]].max()))
    unique_labels = set(train_data_labels)
    print(unique_labels)
    train_data_sizes = []
    train_data_coords = []
    # Import `tensorflow` 
    import tensorflow as tf
    print("before TF modal")

    # Initialize placeholders 
    x = tf.placeholder(dtype = tf.float32, shape = [None, 32, 32])
    y = tf.placeholder(dtype = tf.int32, shape = [None])
    # Flatten the input data
    train_images_flat = tf.contrib.layers.flatten(x)
    # Fully connected layer 
    
    # Input Layer
    input_layer = tf.reshape(train_images_flat, [-1, 32, 32, 1])
    print("conv1")
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="VALID",
        activation=tf.nn.relu)
    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=32,
        kernel_size=[5, 5],
        padding="VALID",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    #Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 5 * 5 * 32])

    # Dense Layer
    dense1 = tf.layers.dense(inputs=pool2_flat, units=256, activation=tf.nn.relu)
    drop_out1 = tf.nn.dropout(dense1, 0.5)
    dense2 = tf.layers.dense(inputs=drop_out1, units=128, activation=tf.nn.relu)
    drop_out2 = tf.nn.dropout(dense2, 0.5)
    dense3 = tf.layers.dense(inputs=drop_out2, units=64, activation=tf.nn.relu)
    drop_out3 = tf.nn.dropout(dense3, 0.5)
    logits = tf.contrib.layers.fully_connected(drop_out3, 43, tf.nn.relu)
    # Define a loss function
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, 
                                                                        logits = logits))
    # Define an optimizer 
    train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    # Convert logits to label indexes
    correct_pred = tf.argmax(logits, 1)
    # Define an accuracy metric
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    saver = tf.train.Saver()
    
    #batch = tf.train.batch([image, label], batch_size=100)
    tf.set_random_seed(1234)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    batch_size = 2000
    hm_epochs = 5000
    for epoch in range(1,hm_epochs+1):
        for j in range(int(34799/batch_size)):
            #print(j)
            #print('    Batch', j+1)
            m=j*batch_size
            n=(j+1)*batch_size
            _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: train_images[m:n], y: train_data_labels[m:n]})

        print('Epoch', epoch, 'completed out of',hm_epochs)
        
        if epoch % 10000 == 0:
            # Run predictions against the full test set.
            predicted_train = sess.run([correct_pred], feed_dict={x: train_images})[0]
            # Calculate correct matches
            match_count_train = sum([int(y == y_) for y, y_ in zip(train_data_labels, predicted_train)])
            # Calculate the accuracy
            accuracy_train = match_count_train / len(train_data_labels)
            # Print the accuracy
            print("Accuracy_train: {:.3f}".format(accuracy_train))
                
    saver.save(sess, './lenet')
    print("Model saved")
    
    import random
    # Pick 10 random train_images
    sample_indexes = random.sample(range(len(train_images)), 10)
    sample_train_images = [train_images[i] for i in sample_indexes]
    sample_labels = [train_data_labels[i] for i in sample_indexes]
    
    """
    #names = imagenet.create_readable_names_for_imagenet_labels()
    for i in range(5):
        index = sorted_inds[i]
        # Now we print the top-5 predictions that the network gives us with
        # corresponding probabilities. Pay attention that the index with
        # class names is shifted by 1 -- this is because some networks
        # were trained on 1000 classes and others on 1001. VGG-16 was trained
        # on 1000 classes.
        print('Probability %0.2f => [%s]' % (probabilities[index]))
    """
    
    # Run the "correct_pred" operation
    predicted = sess.run([correct_pred], feed_dict={x: sample_train_images})[0]
    # Print the real and predicted labels
    print(sample_labels)
    print(predicted)
    
    # Display the predictions and the ground truth visually.
    #fig = plt.figure(figsize=(10, 10))
    for i in range(len(sample_train_images)):
        truth = sample_labels[i]
        prediction = predicted[i]
        plt.subplot(5, 2,1+i)
        plt.axis('off')
        color='green' if truth == prediction else 'red'
        plt.text(40, 10, "Truth:        {0}\nPrediction: {1}".format(truth, prediction), 
                 fontsize=12, color=color)
        plt.imshow(sample_train_images[i], cmap="gray")
    
    plt.show()
    ###########################################################
    
    """
    # Run predictions against the full test set.
    predicted_train = sess.run([correct_pred], feed_dict={x: train_images})[0]
    # Calculate correct matches 
    match_count_train = sum([int(y == y_) for y, y_ in zip(train_data_labels, predicted_train)])
    # Calculate the accuracy
    accuracy_train = match_count_train / len(train_data_labels)
    # Print the accuracy
    print("Accuracy_train: {:.3f}".format(accuracy_train))
    """
    
    train_data_sizes = []
    train_data_coords = []
    train_data_features = []
    train_data_labels = []
    
    #crop(train_images[1],train_data_coords[1])
    valid_data_sizes = []
    valid_data_coords = []
    valid_data_features = []
    valid_data_labels = []
    try:
        with open('F:/Udacity/Project 2/traffic-signs-data/valid.p','rb') as snapFile:
        #with open('C:/Users/alsshxu/Desktop/2018 HX/UdaCity/Project 2/traffic-signs-data/valid.p','rb') as snapFile:
            savedSnaps = pickle.load(snapFile)
    except IOError as e:
        return False
    #print(savedSnaps)
    valid_data_sizes=savedSnaps['sizes']
    valid_data_coords = savedSnaps['coords']
    valid_data_features = savedSnaps['features']
    valid_data_labels = savedSnaps['labels']
    
    valid_images = rgb2gray(valid_data_features)
    for i in range(0,4410):
        valid_data_coords[i][0]=int(valid_data_coords[i][0] * 32 / valid_data_sizes[i][0])
        valid_data_coords[i][1]=int(valid_data_coords[i][1] * 32 / valid_data_sizes[i][1])
        valid_data_coords[i][2]=int(valid_data_coords[i][2] * 32 / valid_data_sizes[i][0])+1
        valid_data_coords[i][3]=int(valid_data_coords[i][3] * 32 / valid_data_sizes[i][1])+1
        crop_img = valid_images[i][valid_data_coords[i][1]:valid_data_coords[i][3], valid_data_coords[i][0]:valid_data_coords[i][2]]
        valid_images[i] = cv2.resize(crop_img,(32,32))
    
    a1,b1 = np.shape(valid_data_sizes)
    print (a1)
    print (b1)
    
    predicted_valid = sess.run([correct_pred], feed_dict={x: valid_images})[0]
    # Calculate correct matches 
    match_count_valid = sum([int(y == y_) for y, y_ in zip(valid_data_labels, predicted_valid)])
    # Calculate the accuracy
    accuracy_valid = match_count_valid / len(valid_data_labels)
    # Print the accuracy
    print("Accuracy_valid: {:.3f}".format(accuracy_valid))
    
    valid_data_sizes = []
    valid_data_coords = []
    valid_data_features = []
    valid_data_labels = []

    test_data_sizes = []
    test_data_coords = []
    test_data_features = []
    test_data_labels = []
    try:
        with open('F:/Udacity/Project 2/traffic-signs-data/test.p','rb') as snapFile:
        #with open('C:/Users/alsshxu/Desktop/2018 HX/UdaCity/Project 2/traffic-signs-data/test.p','rb') as snapFile:
            savedSnaps = pickle.load(snapFile)
    except IOError as e:
        return False
    #print(savedSnaps)
    test_data_sizes=savedSnaps['sizes']
    test_data_coords = savedSnaps['coords']
    test_data_features = savedSnaps['features']
    test_data_labels = savedSnaps['labels']
    test_images = rgb2gray(test_data_features)
    
    a1,b1,c1 = np.shape(test_images)
    print (a1)
    print (b1)
    print (c1)
    test_data_sizes[2847][0]=229
    test_data_sizes[6373][0]=232
    test_data_sizes[6612][0]=225
    
    for i in range(0,12630):
        test_data_coords[i][0]=int(test_data_coords[i][0] * 32 / test_data_sizes[i][0])
        test_data_coords[i][1]=int(test_data_coords[i][1] * 32 / test_data_sizes[i][1])
        test_data_coords[i][2]=int(test_data_coords[i][2] * 32 / test_data_sizes[i][0])+1
        test_data_coords[i][3]=int(test_data_coords[i][3] * 32 / test_data_sizes[i][1])+1
        crop_img = test_images[i][test_data_coords[i][1]:test_data_coords[i][3], test_data_coords[i][0]:test_data_coords[i][2]]

        test_images[i] = cv2.resize(crop_img,(32,32))

    predicted_test = sess.run([correct_pred], feed_dict={x: test_images})[0]
    
    # Calculate correct matches 
    match_count_test = sum([int(y == y_) for y, y_ in zip(test_data_labels, predicted_test)])
    # Calculate the accuracy
    accuracy_test = match_count_test / len(test_data_labels)
    # Print the accuracy
    print("Accuracy_test: {:.3f}".format(accuracy_test))

    test_data_sizes = []
    test_data_coords = []
    test_data_features = []
    test_data_labels = []

    import glob

    my_images = sorted(glob.glob('./web image/*.png'))
    my_labels = np.array([37, 18, 1, 22, 35, 15])

    figures = {}
    labels = {}
    my_signs = []
    index = 0
    for my_image in my_images:
        img = cv2.cvtColor(cv2.imread(my_image), cv2.COLOR_BGR2RGB)
        my_signs.append(img)
        figures[index] = img
        labels[index] = name_values[my_labels[index]][1].decode('ascii')
        index += 1
        
    def plot_figures(figures, nrows = 1, ncols=1, labels=None):
        fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(12, 14))
        axs = axs.ravel()
        for index, title in zip(range(len(figures)), figures):
            axs[index].imshow(figures[title], plt.gray())
            if(labels != None):
                axs[index].set_title(labels[index])
            else:
                axs[index].set_title(title)
            axs[index].set_axis_off()

        plt.tight_layout()
    
    my_signs = np.array(my_signs)
    my_signs_normalized = rgb2gray(my_signs)
    n_train = my_signs.shape[1:]
    print(n_train)
    
    m_train = my_signs_normalized.shape[1:]
    print(m_train)
    
    k_size = 5
    softmax_logits = tf.nn.softmax(logits)
    top_k = tf.nn.top_k(softmax_logits, k=k_size)
    keep_prob = tf.placeholder(tf.float32)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, "./lenet")
        
        predicted = sess.run([correct_pred], feed_dict={x: my_signs_normalized})[0]
        # Print the real and predicted labels
        print(my_labels)
        print(predicted)
        my_top_k = sess.run(top_k, feed_dict={x: my_signs_normalized, keep_prob: 1.0})

        for i in range(6):
            figures = {}
            labels = {}
            figures[0] = my_signs[i]
            labels[0] = "Original"
            print('Image {} :'.format(i+1))
            for j in range(k_size):
                print('    Possibility {} : ({:.0f}%)'.format(j+1, 100*my_top_k[0][i][j]))
                labels[j+1] = 'Guess {} : ({:.0f}%)'.format(j+1, 100*my_top_k[0][i][j])
    return savedSnaps

Main()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

shape: (32, 32), min: 0.09213058823529413, max: 0.41472451439950986


<Figure size 640x480 with 1 Axes>

shape: (32, 32), min: 0.05897559627757353, max: 0.23410500268075982


...

Epoch 4506 completed out of 5000
Epoch 4507 completed out of 5000
Epoch 4508 completed out of 5000
Epoch 4509 completed out of 5000
Epoch 4510 completed out of 5000
Epoch 4511 completed out of 5000
Epoch 4512 completed out of 5000
Epoch 4513 completed out of 5000
Epoch 4514 completed out of 5000
Epoch 4515 completed out of 5000
Epoch 4516 completed out of 5000
Epoch 4517 completed out of 5000
Epoch 4518 completed out of 5000
Epoch 4519 completed out of 5000
Epoch 4520 completed out of 5000
Epoch 4521 completed out of 5000
Epoch 4522 completed out of 5000
Epoch 4523 completed out of 5000
Epoch 4524 completed out of 5000
Epoch 4525 completed out of 5000
Epoch 4526 completed out of 5000
Epoch 4527 completed out of 5000
Epoch 4528 completed out of 5000
Epoch 4529 completed out of 5000
Epoch 4530 completed out of 5000
Epoch 4531 completed out of 5000
Epoch 4532 completed out of 5000
Epoch 4533 completed out of 5000
Epoch 4534 completed out of 5000
Epoch 4535 completed out of 5000
Epoch 4536

Epoch 4755 completed out of 5000
Epoch 4756 completed out of 5000
Epoch 4757 completed out of 5000
Epoch 4758 completed out of 5000
Epoch 4759 completed out of 5000
Epoch 4760 completed out of 5000
Epoch 4761 completed out of 5000
Epoch 4762 completed out of 5000
Epoch 4763 completed out of 5000
Epoch 4764 completed out of 5000
Epoch 4765 completed out of 5000
Epoch 4766 completed out of 5000
Epoch 4767 completed out of 5000
Epoch 4768 completed out of 5000
Epoch 4769 completed out of 5000
Epoch 4770 completed out of 5000
Epoch 4771 completed out of 5000
Epoch 4772 completed out of 5000
Epoch 4773 completed out of 5000
Epoch 4774 completed out of 5000
Epoch 4775 completed out of 5000
Epoch 4776 completed out of 5000
Epoch 4777 completed out of 5000
Epoch 4778 completed out of 5000
Epoch 4779 completed out of 5000
Epoch 4780 completed out of 5000
Epoch 4781 completed out of 5000
Epoch 4782 completed out of 5000
Epoch 4783 completed out of 5000
Epoch 4784 completed out of 5000
Epoch 4785

<Figure size 640x480 with 10 Axes>

4410
2
Accuracy_valid: 0.951
12630
32
32
Accuracy_test: 0.930
(32, 32, 3)
(32, 32)
INFO:tensorflow:Restoring parameters from ./lenet
[37 18  1 22 35 15]
[37 18  1 22 35 15]
Image 1 :
    Possibility 1 : (100%)
    Possibility 2 : (0%)
    Possibility 3 : (0%)
    Possibility 4 : (0%)
    Possibility 5 : (0%)
Image 2 :
    Possibility 1 : (100%)
    Possibility 2 : (0%)
    Possibility 3 : (0%)
    Possibility 4 : (0%)
    Possibility 5 : (0%)
Image 3 :
    Possibility 1 : (100%)
    Possibility 2 : (0%)
    Possibility 3 : (0%)
    Possibility 4 : (0%)
    Possibility 5 : (0%)
Image 4 :
    Possibility 1 : (95%)
    Possibility 2 : (3%)
    Possibility 3 : (2%)
    Possibility 4 : (0%)
    Possibility 5 : (0%)
Image 5 :
    Possibility 1 : (100%)
    Possibility 2 : (0%)
    Possibility 3 : (0%)
    Possibility 4 : (0%)
    Possibility 5 : (0%)
Image 6 :
    Possibility 1 : (100%)
    Possibility 2 : (0%)
    Possibility 3 : (0%)
    Possibility 4 : (0%)
    Possibility 5 : (0%)


{'sizes': array([[53, 54],
        [42, 45],
        [48, 52],
        ...,
        [29, 29],
        [48, 49],
        [32, 31]], dtype=uint8), 'coords': array([[ 3,  2, 29, 30],
        [ 3,  3, 28, 29],
        [ 4,  3, 29, 29],
        ...,
        [ 6,  6, 27, 27],
        [ 3,  3, 29, 29],
        [ 6,  5, 28, 27]], dtype=uint8), 'features': array([[[[116, 139, 174],
          [116, 137, 171],
          [118, 138, 172],
          ...,
          [ 98, 114, 143],
          [ 97, 121, 147],
          [ 85, 105, 130]],
 
         [[115, 142, 176],
          [115, 140, 174],
          [117, 141, 175],
          ...,
          [119, 143, 176],
          [120, 141, 173],
          [120, 139, 171]],
 
         [[117, 141, 174],
          [117, 142, 175],
          [114, 140, 172],
          ...,
          [120, 144, 179],
          [122, 144, 179],
          [120, 143, 177]],
 
         ...,
 
         [[117, 137, 167],
          [115, 134, 164],
          [117, 135, 164],
          ...,